# **Investment Portfolio Management**

## **About the scenario**
This scenario demonstrates best practices in a data engineering context, where notebooks are used to orchestrate complex tasks involving real-time data fetching, computation, and report delivery. The notebook is modular, and secure, employing batch processing, error handling, and proper separation of concerns.

In this scenario, we will:

1. Ingest a CSV file containing the user’s investment portfolio.
2. Fetch real-time stock prices via an API using *Function Calling*.
3. Perform calculations on the portfolio, including total value and percentage changes, using *Code Interpreter*.
4. Generate a detailed report summarizing the portfolio's performance.
5. Save the report to Azure Blob Storage using *Function Calling*.

## **Time**
You should expect to spend 10-15 minutes building and running this scenario. 

## **Before you begin**

#### Step 1: Install required libraries
Including a cell to install dependencies directly within a Jupyter notebook is a good practice because it ensures that all required packages are installed in the correct versions, making the notebook self-contained and reproducible. This approach helps other users or collaborators to set up the environment quickly and avoid potential issues related to missing or incompatible packages.

In [ ]:
# Install the packages
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### Step 2: Setting up the environment

Before we begin, we'll load necessary environment variables from a `.env` file. These variables contain sensitive information such as API keys and endpoint URLs. Ensure your `.env` file is properly configured in the `.venv/.env` format. 

In [202]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path=".venv/.env")

# Retrieve the secrets
__AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
__AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
__AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
__AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# Verify environment variables
if not all([__AZURE_OPENAI_API_KEY, __AZURE_OPENAI_ENDPOINT, __AZURE_OPENAI_DEPLOYMENT, __AZURE_OPENAI_API_VERSION]):
    raise EnvironmentError("One or more environment variables are missing. Please check the .env file.")

print("Environment variables loaded successfully.")

Environment variables loaded successfully.


## **Azure OpenAI setup**

#### Step 1: Initializing the Azure OpenAI Client

Next, we’ll initialize the Azure OpenAI client. This client allows us to interact with Azure OpenAI's file management APIs. You’ll need your API key, endpoint, and API version, which we just loaded from the environment.

In [203]:
from openai import AzureOpenAI, OpenAIError

# Initialize the AzureOpenAI client
try:
    openai_client = AzureOpenAI(
        api_key=__AZURE_OPENAI_API_KEY,
        api_version=__AZURE_OPENAI_API_VERSION,
        azure_endpoint=__AZURE_OPENAI_ENDPOINT
    )
    print("AzureOpenAI client initialized.")
except OpenAIError as e:
    raise ConnectionError(f"Failed to initialize AzureOpenAI client: {e}")


AzureOpenAI client initialized.


#### Step 2: Upload supporting file to Azure OpenAI deployment

Now, we'll upload the `investment_portfolio.csv` file from the `\data` directory to Azure OpenAI, ensuring any duplicates are removed beforehand. This process will handle the entire upload. The file is necessary for this scenario, but its contents can be modified as long as the file structure remains unchanged.

In [ ]:
# Directory containing files to upload
directory="data"
portfolio_file="investment_portfolio.csv"

# Check if the directory exists
if not os.path.isdir(directory):
    print(f"Directory '{directory}' does not exist.")
    raise FileNotFoundError(f"Directory '{directory}' does not exist.")

file_path = os.path.join(directory, portfolio_file)

# Check if the file exists
if not os.path.isfile(file_path):
    print(f"Skipping non-file item: {portfolio_file}")

try:
    # Delete existing file on Azure if it has the same name and purpose
    existing_files = openai_client.files.list()
    for f in existing_files:
        if f.filename == portfolio_file and f.purpose == "assistants":
            openai_client.files.delete(file_id=f.id)
            print(f"Deleted existing file: {portfolio_file}")

    # Upload new file
    with open(file_path, "rb") as file_data:
        openai_client.files.create(file=file_data, purpose="assistants")
    print(f"Uploaded file: {portfolio_file}")

except OpenAIError as e:
    print(f"Error processing file '{portfolio_file}': {e}")
except Exception as e:
    print(f"Unexpected error with file '{portfolio_file}': {e}")

Deleted existing file: investment_portfolio.csv
Uploaded file: investment_portfolio.csv


## **Azure OpenAI Assistant**

### Step 1: Define function for Assistant
The `fetch_stock_price` function retrieves the stock data for a specified `ticker symbol` using the `yfinance` library, specifically pulling the latest data for the last trading day.

In [228]:
import yfinance as yf

def fetch_stock_price(ticker_symbol):
    """
    Fetch the latest stock price for a given ticker symbol.

    Parameters:
    - ticker_symbol (str): The ticker symbol of the stock to retrieve data for.

    Returns:
    - str: The closing price of the stock for the latest trading day, or an error message if data is unavailable.

    Example:
    >>> fetch_stock_price("AAPL")
    "148.9"
    """
    
    try:
        # Fetch the stock's trading history for the last day
        stock = yf.Ticker(ticker_symbol)
        stock_data = stock.history(period="1d")

        # Check if the data is empty, indicating an invalid ticker or no data available
        if stock_data.empty:
            return f"Error: No data found for ticker symbol: {ticker_symbol}"

        # Retrieve and return the latest closing price
        latest_close_price = stock_data['Close'].iloc[-1]
        return str(round(latest_close_price, 3))

    except KeyError as e:
        return f"Error: Data missing for key: {e}. Verify the ticker symbol."

    except Exception as e:
        return f"Error: Unexpected issue occurred - {type(e).__name__}: {e}"
    
print("Function defined successfully.")

Function defined successfully.


### Step 2: Define Function Calling tool for Assistant
Create the `fetch_stock_price` function configuration to be used within the code interpreter. By adding this tool to the Assistant, we enable it to retrieve and interpret stock data when a user prompts it with related queries.

In [206]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "fetch_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            "parameters": {
                "type": "object",
                "properties": {"ticker_symbol": {"type": "string", "description": "The ticker symbol of the stock"}},
                "required": ["ticker_symbol"],
            },
        },
    }
]

print("Tools list defined successfully.")

Tools list defined successfully.


### Step 3: Creating the Investment Management Assistant
In this step, we create an assistant equipped with specialized tools, including a code interpreter, to handle investment-related queries and utilize the previously defined function calls. This assistant will analyze the uploaded portfolio file and offer valuable insights.

In [230]:
# Create the assistant with code interpreter enabled
try:
    assistant = openai_client.beta.assistants.create(
        name="Investment Management Assistant",
        instructions=(
            "You are an expert investment analyst. "
            "Use your knowledge base to answer questions about personal investment portfolio management."
        ),
        model=__AZURE_OPENAI_DEPLOYMENT,
        tools=tools_list
    )
    print("Assistant created successfully.")
except OpenAIError as e:
    print("Error creating assistant:", e)

Assistant created successfully.


## **Query Assistant**

### Step 1: Starting a New Conversation Thread
Let's create a new thread to handle user interactions. Each thread allows for a dedicated conversation with the assistant.

In [208]:
# Create a conversation thread
try:
    thread = openai_client.beta.threads.create()
    print("Thread created:", thread)
except OpenAIError as e:
    print("Error creating thread:", e)

Thread created: Thread(id='thread_5vg4RVr6m1PLLD5LwfNhdtMg', created_at=1731527292, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Step 2: Adding User Message
In this step, we add a question to the thread. For this demonstration, we're asking which stock ticker has the most investment in the portfolio.

In [209]:
# Define the user question
#prompt_content = "What ticker symbol has the most stock? Which has the most investment?"

#prompt_content = "What is the latest closing price of AAPL?"

prompt_content = "What is the latest closing price for MSFT? What is my total investment for MSFT as of today?"

# Add the question to the thread
try:
    message = openai_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt_content,
        attachments=[  # Add files by using the attachments parameter
            {"file_id": portfolio_file.id, "tools": [{"type": "code_interpreter"}]}
        ],
    )
    print("User question added:", message)
except OpenAIError as e:
    print("Error adding user question:", e)


User question added: Message(id='msg_JeT18F1yiXMnjQpDWEeUXHwE', assistant_id=None, attachments=[Attachment(file_id='assistant-cOAyVn8FSeCER5PL0ruwAOK9', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the latest closing price for MSFT? What is my total investment for MSFT as of today?'), type='text')], created_at=1731527292, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_5vg4RVr6m1PLLD5LwfNhdtMg')


### Step 3: Running the Assistant
Now that the assistant and thread are set up, we'll initiate the assistant's response process. This will analyze the user prompt and provide insights based on the investment portfolio data.


In [210]:
# Initiate the assistant's response
try:
    run = openai_client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=prompt_content,
    )
    print("Run started:", run)
except OpenAIError as e:
    print("Error starting run:", e)

Run started: Run(id='run_emdA243wUC3jMMwwfywNcfWM', assistant_id='asst_dy9pQwPHiMh5iw2URC1eA5j8', cancelled_at=None, completed_at=None, created_at=1731527293, expires_at=1731527893, failed_at=None, incomplete_details=None, instructions='What is the latest closing price for MSFT? What is my total investment for MSFT as of today?', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_5vg4RVr6m1PLLD5LwfNhdtMg', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='fetch_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'ticker_symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['ticker_symbol']}, strict=False), type='function')], t

 Define the dictionary `available_functions` that maps function names to their corresponding implementations, in this case, fetch_stock_price

In [211]:
available_functions = {"fetch_stock_price": fetch_stock_price}

### Step 4: Monitor run status

The assistant may take some time to analyze and respond. We'll use a polling loop to periodically check the assistant's status and retrieve the answer once it's available.
Generally we use a loop to check the status and perform actions based on the outcome of each check. To understand the status actions more, lets simulate a loops. Click the next 4 cells multiple times to see the progress of the Assitant Run.
Retrieves the current status of the OpenAI Assistant run. The status is printed in a formatted JSON string for clarity. Depending on the status of the run, *different actions are taken*.

*Status: Queued*
What does this mean - probably wont use. but explain

In [221]:
import json

# Retrieve the run status
run = openai_client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
#print(run.model_dump_json(indent=4))


if run.status in ('queued', 'in_progress'):
    print(f"Run this cell again to monitor the status.\n Current Status: {run.status}")
else:
    print(f"Monitoring the run status...\nCurrent Status: {run.status}")

Monitoring the run status...
Current Status: completed


#### Step 4a: Failed status

IF the status is `failed` we will print the error message with relevant information to aid in troubleshooting.

In [213]:
#print(run.model_dump_json(indent=4))

if run.status == "failed":
    print("Assistant run failed. Please try again.")
else:
    print(f"Assistant run has not failed...\nNavigate to and execute the '{run.status}' cell.")


Assistant run has not failed...
Navigate to and execute the 'queued' cell.


#### Step 4b: Requires Action status

If the status is `requires_action`, first we need to check if the required action is to submit tool outputs and iterate over the tool calls, ensuring that the requested function exists in the available_functions dictionary. If the function exists, it is called with the provided arguments, and the response is stored. After processing all tool calls, we submit the tool outputs back to the OpenAI client.

In [219]:
if run.status == "requires_action":
    print("Function Calling ...")
    tool_responses = []
    if (
        run.required_action.type == "submit_tool_outputs"
        and run.required_action.submit_tool_outputs.tool_calls is not None
    ):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls

        for call in tool_calls:
            if call.type == "function":
                if call.function.name not in available_functions:
                    raise Exception("Function requested by the model does not exist")
                function_to_call = available_functions[call.function.name]
                tool_response = function_to_call(**json.loads(call.function.arguments))
                tool_responses.append({"tool_call_id": call.id, "output": tool_response})
                print(f"Function '{call.function.name}' called successfully. \nOutput: {tool_response}\n")

    run = openai_client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
    )

    print(f"Results submitted successfully. Go back to the first cell, 'Monitor Run Status' and execute again.")
else:
    print(f"Navigate to and execute the {run.status} cell.")

Function Calling ...
Function 'fetch_stock_price' called successfully. 
Output: 426.48

Results submitted successfully. Go back to the first cell, 'Monitor Run Status' and execute again.


#### Step 4c: Completed status

If the status is `completed`, we will fetch and print all messages in the thread, displaying the role and content of each message.The messages are printed in reverse order because messages in a thread are in FILO (First-In-Last-Out) order and in order to make the messages more conversational for ease of user reading, we must reverse the order.

In [227]:
if run.status == "completed":
    messages = openai_client.beta.threads.messages.list(thread_id=thread.id, order="asc", after=message.id)

    print(f'Run completed!\n\nMESSAGES\n')

    # Loop through messages and print content based on role
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")

else:
    print(f"Navigate to and execute the {run.status} cell.")



Run completed!

MESSAGES

Assistant: The latest closing price for MSFT (Microsoft) is $426.48.

Now, I will calculate your total investment in MSFT. I will first inspect the contents of the file you uploaded.
Assistant: The file contains information about your investments. Here are the relevant columns:
- Symbol: The ticker symbol of the stock
- Average_Cost: The average cost price of the stock
- QTY: The quantity of the stock you own

Let's calculate your total investment in MSFT as of today.
Assistant: Your total investment in MSFT as of today is $127,944.00.


#### Next Step / Let's refine 
Wrap in function - explain the function

In [216]:
"""

import json
import time


available_functions = {"fetch_stock_price": fetch_stock_price}

while True:

    # Retrieve the run status
    run = openai_client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run.model_dump_json(indent=4))

    if run.status == "failed":
         print("Assistant run failed. Please try again.")
         break
    elif run.status == "completed":
        messages = openai_client.beta.threads.messages.list(thread_id=thread.id, order="asc")

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")
        break
    elif run.status == "requires_action":
        print("Function Calling ...")
        tool_responses = []
        if (
            run.required_action.type == "submit_tool_outputs"
            and run.required_action.submit_tool_outputs.tool_calls is not None
        ):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls

            for call in tool_calls:
                if call.type == "function":
                    if call.function.name not in available_functions:
                        raise Exception("Function requested by the model does not exist")
                    function_to_call = available_functions[call.function.name]
                    tool_response = function_to_call(**json.loads(call.function.arguments))
                    tool_responses.append({"tool_call_id": call.id, "output": tool_response})
                    print(f"Function '{call.function.name}' called successfully. \nOutput: {tool_response}\n")

        run = openai_client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
        )

        print("Function called successfully.")
    else:
            print("Waiting for the Assistant to process...")
            time.sleep(5)
"""

'\n\nimport json\nimport time\n\n\navailable_functions = {"fetch_stock_price": fetch_stock_price}\n\nwhile True:\n\n    # Retrieve the run status\n    run = openai_client.beta.threads.runs.retrieve(\n        thread_id=thread.id,\n        run_id=run.id\n    )\n    print(run.model_dump_json(indent=4))\n\n    if run.status == "failed":\n         print("Assistant run failed. Please try again.")\n         break\n    elif run.status == "completed":\n        messages = openai_client.beta.threads.messages.list(thread_id=thread.id, order="asc")\n\n        # Loop through messages and print content based on role\n        for msg in messages.data:\n            role = msg.role\n            content = msg.content[0].text.value\n            print(f"{role.capitalize()}: {content}")\n        break\n    elif run.status == "requires_action":\n        print("Function Calling ...")\n        tool_responses = []\n        if (\n            run.required_action.type == "submit_tool_outputs"\n            and run.

#### More Examples using Function
- one basic with image (uses CI)
- one basic function calling 

### Wrap Up

## **Cleanup Resources**
To avoid creating redundant resources and ensure a clean environment, this cell deletes the assistant, thread, and any other created resources. Run this cell at the end of your session to clean up.


In [217]:
#response = openai_client.beta.assistants.delete(assistant.id)
#print(response)

# Optionally delete any other temporary files or datas
# Note: Any uploaded files to OpenAI could also be cleaned up if needed
